In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time


# Import API key
from pprint import pprint
from config import api_key1 
from pandas.io.json import json_normalize

import urllib3
urllib3.disable_warnings()


In [28]:
#starting URL

keywords =input("enter search criteria")
url = "https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=" + api_key1 + "&keyword="+keywords+"&radius=300"

print(url)

enter search criteriaFlorida Gang
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=2eef5d73f1bb2c6b0aba8aca93436ea0f56df309&keyword=Florida Gang&radius=300


In [38]:

titles_1 = []
summary_1 = []
link_1 = []
time_1 = []
source_1 = []
address_1 = []
category_1 = []

articles = requests.get(url,verify=False).json()
print(articles['next'])
i = 0
while True:
   
    for result in articles['results'] :
        titles_1.append(result['title'])
        summary_1.append(result['summary'])
        time_1.append(result['time'])
        link_1.append(result['link'])
        source_1.append(result['source']['name'])
        address_1.append(result['source']['address'])
        category_1.append("Couple")
        i = i + 1
    if i >1000000: 
        break
    articles = requests.get(articles['next'],verify=False).json()
        
    if articles == {}:
        break
            
    print(articles['next'])
    

https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=2eef5d73f1bb2c6b0aba8aca93436ea0f56df309&keyword=Florida+Gang&radius=300&cursor=AoIIQ9drKzluZXdzZmVlZC5hcnRpY2xlLjc4MDE5NzY2


KeyError: 'results'

In [23]:
news_df_dict = {'Title': titles_1,
          'Summary': summary_1,
          'Link': link_1,
          'Time': time_1,
          'Source': source_1,
            'Category': category_1,   
          'Address': address_1}

news_df = pd.DataFrame(news_df_dict)

news_df.to_csv("data/Local_"+keywords+".csv", index=False)

In [24]:
local_df = pd.read_csv("data/Local_"+keywords+".csv")

In [39]:
local_df = local_df.dropna(how='all')

new = local_df['Address'].str.split(",",n=3, expand=True)
new['street_address'] = new[0]
new['city'] = new[1]
new['state'] = new[2]
new1 = new['state'].str.split(n=1, expand=True)

new1.rename(columns={0: "State",1: "Zip"}, inplace=True)

clean = local_df.merge(new1, right_index=True, left_index=True)

#wh_data_cl = wh_data.loc[wh_data['Humidity'] <= 101]
clean_df = clean.loc[clean['State'] == "FL"]

In [26]:
local_news_df = clean_df
local_news_df.to_csv("data/Local_"+keywords+".csv")